In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import gtfs_kit as gk
import math


In [11]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calcula distância em km usando fórmula de Haversine"""
    R = 6371  # Raio da Terra em km
    
    # Converter graus para radianos
    lat1_rad = math.radians(lat1)
    lat2_rad = math.radians(lat2)
    lon1_rad = math.radians(lon1)
    lon2_rad = math.radians(lon2)
    
    # Diferenças
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Fórmula de Haversine
    a = math.sin(delta_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c

In [12]:
# Função para encontrar paragens próximas
def find_nearby_stops(stcp_df, metro_df, max_distance_km=0.05):
    """Encontra paragens a menos de X km de distância"""
    common_stops = []
    
    for idx_stcp, row_stcp in stcp_df.iterrows():
        lat1 = row_stcp['stop_lat']
        lon1 = row_stcp['stop_lon']
        
        for idx_metro, row_metro in metro_df.iterrows():
            lat2 = row_metro['stop_lat']
            lon2 = row_metro['stop_lon']
            
            # Calcular distância
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            
            if distance <= max_distance_km:
                common_stops.append({
                    'stcp_stop_id': row_stcp['stop_id'],
                    'metro_stop_id': row_metro['stop_id'],
                    'stcp_stop_name': row_stcp['stop_name'],
                    'metro_stop_name': row_metro['stop_name'],
                    'distance_km': distance,
                    'stcp_lat': lat1,
                    'stcp_lon': lon1,
                    'metro_lat': lat2,
                    'metro_lon': lon2
                })
    
    return pd.DataFrame(common_stops)

In [13]:
# Carregar dados GTFS
stcp_path = "dataset/stcp"
metro_path = "dataset/metro_porto"

# Carregar feeds GTFS
stcp_feed = gk.read_feed(stcp_path, dist_units='km')
metro_feed = gk.read_feed(metro_path, dist_units='km')

# Extrair paragens/stops
stcp_stops = stcp_feed.stops.copy()
metro_stops = metro_feed.stops.copy()

# Adicionar identificadores de operador
stcp_stops['operator'] = 'stcp'
metro_stops['operator'] = 'metro'

In [ ]:
# Encontrar paragens comuns (intermodais)
common_stops_df = find_nearby_stops(stcp_stops, metro_stops, max_distance_km=0.05)

In [ ]:


# Marcar paragens comuns em cada dataset
def is_common_stop(row, other_stops, max_distance_km=0.2):
    """Verifica se uma paragem tem correspondente no outro dataset"""
    for _, other_row in other_stops.iterrows():
        distance = haversine_distance(
            row['stop_lat'], row['stop_lon'],
            other_row['stop_lat'], other_row['stop_lon']
        )
        if distance <= max_distance_km:
            return True
    return False

# Marcar paragens STCP que são comuns
stcp_stops['is_common'] = stcp_stops.apply(
    lambda row: is_common_stop(row, metro_stops, max_distance_km=0.2), 
    axis=1
)

# Marcar paragens Metro que são comuns
metro_stops['is_common'] = metro_stops.apply(
    lambda row: is_common_stop(row, stcp_stops, max_distance_km=0.2), 
    axis=1
)

# Criar lista de aragens comuns (intermodais)
common_stops_ids = {
    'stcp': set(common_stops_df['stcp_stop_id'].tolist()),
    'metro': set(common_stops_df['metro_stop_id'].tolist())
}

# Juntar todas as paragens
all_stops = pd.concat([stcp_stops, metro_stops], ignore_index=True)

# Extrair rotas e serviços
stcp_routes = stcp_feed.routes.copy()
metro_routes = metro_feed.routes.copy()




=== ESTATÍSTICAS DAS PARAGENS ===
Número total de paragens (incluindo duplicadas): 2587

Paragens STCP exclusivas: 2290
Paragens Metro exclusivas: 28
Paragens comuns (intermodais): 212

Verificação: STCP(2502) + Metro(240) = Total(2587)

=== ESTATÍSTICAS DAS ROTAS ===
Número total de rotas STCP: 73
Número total de rotas Metro: 7
Número total de rotas multimodais: 80

=== EXEMPLO DE PARAGENS INTERMODAIS (primeiras 5) ===

Paragem 1:
  STCP: CONTUMIL (METRO) (ID: CNTM2)
  Metro: Contumil (ID: 5707)
  Distância: 32 metros

Paragem 2:
  STCP: CAMPO 24 DE AGOSTO (ID: C24A3)
  Metro: 24 de Agosto (ID: 5697)
  Distância: 42 metros

Paragem 3:
  STCP: STO. OVÍDIO (ID: SO3)
  Metro: Santo Ovídio (ID: 5792)
  Distância: 42 metros

Paragem 4:
  STCP: SENHOR MATOSINHOS (ID: SMAT1)
  Metro: Senhor de Matosinhos (ID: 5723)
  Distância: 14 metros

Paragem 5:
  STCP: GENERAL TORRES (ID: GENT2)
  Metro: General Torres (ID: 5771)
  Distância: 13 metros

=== RESUMO ===
            Tipo  Contagem
 STCP Ex